In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2023-05-01


Found rundate 2023-04-30 10:00:03 < 2023-05-01 at commit e4b49d88; returning commit c36cf65


'2023-05-01 10:00:02'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2023-05-01 10:00:02, NJSP was reporting 155 YTD deaths
Current YTD Deaths (2024-05-01 10:00:03-04:00): 201
Previous year YTD Deaths (adjusted; 2023-05-01 10:00:02): 155.86064198114335
Projected 2024 total: 695.4682832336625
29.0% change


FAUQStats cache miss: 2023, Mon May 01 10:00:02 EDT 2023


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12249,07,Essex,0714,Newark City,Mulberry St,Mulberry St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-01 01:49:00-05:00
12251,01,Atlantic,0112,Hamilton Twsp,NaN,State Highway 322 MP 46.4,1.0,1.0,0.0,0.0,0.0,322,0.0,2023-01-02 09:26:00-05:00
12253,12,Middlesex,1217,Piscataway Twsp,South Randolphville Rd,South Randolphville Rd,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2023-01-02 19:41:00-05:00
12254,02,Bergen,0221,Garfield City,Outwater Ln,Outwater Ln,1.0,0.0,0.0,1.0,0.0,NaN,0.0,2023-01-03 20:52:00-05:00
12267,07,Essex,0714,Newark City,Washington St,Washington St,1.0,0.0,0.0,1.0,0.0,NaN,NaN,2023-01-04 06:21:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12424,03,Burlington,0329,Pemberton Boro,NaN,County 687 MP .25,1.0,1.0,0.0,0.0,0.0,687,1.0,2023-04-20 17:16:00-04:00
12425,15,Ocean,1520,Ocean Twsp,Bay Pkwy,Bay Pkwy,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2023-04-22 14:39:00-04:00
12427,11,Mercer,1103,Hamilton Twsp,NaN,County 533,1.0,1.0,0.0,0.0,0.0,533,NaN,2023-04-22 16:45:00-04:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,7,1,5,1,14
Bergen,7,2,9,0,18
Burlington,9,2,2,0,13
Camden,6,2,9,0,17
Cape May,1,0,0,0,1
Cumberland,3,1,1,0,5
Essex,5,3,5,0,13
Gloucester,5,2,0,0,7
Hudson,2,2,1,0,4


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

33.2% through the year, 66.8% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

prv_ytd  prv_end  cur_ytd
county     type                                 
Atlantic   driver            5       17        7
Bergen     driver            4       20        7
Burlington driver            3       26        9
Camden     driver            5       18        6
Cape May   driver            2        4        1
...                        ...      ...      ...
Sussex     driver            0        6        1
           passenger         0        2        0
           pedestrian        0        1        1
           cyclist           0        0        0
           crashes           0        6        2

[105 rows x 3 columns]

In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,41,3,21,7,13
Bergen,47,1,27,6,14
Burlington,52,1,47,5,4
Camden,61,3,20,7,46
Cape May,5,0,3,1,1
Cumberland,18,0,13,4,3
Essex,50,1,18,6,24
Gloucester,32,1,27,6,2
Hudson,19,1,10,5,6


In [9]:
projected.to_csv(PROJECTED_CSV)